In [1]:
import pandas as pd
import numpy as np  
import re
import sys

import PyUber

import PRODUCT as prod
import REDWING as redwing

In [2]:
rw = redwing.RedWing(
    xeus_source='F32_PROD_XEUS',
    debug_flag=True)

d:\Python\NPI\REDWING.py:67: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3]:
NPI = rw.NPIs[2]
match = re.search(r'\((.*?)\)', NPI)
NPI_product = match.group(1) if match else None
NPI_product = NPI_product.replace(' ','%')
print(NPI_product)

8PXMCV%A%G


In [4]:
prod1 = prod.Product(npi=NPI,
                product=NPI_product,
                xeus_source = 'F32_PROD_XEUS',
                debug_flag = True)

d:\Python\NPI\PRODUCT.py:102: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Running master query for LOT: 'A528013'


In [5]:
# Future add_lot code def

rw.NPI_List

lots = rw.get_lots(GROUP_NAME=NPI)
print(lots)
# for lot in lots:
    # print(lot)

# lot_index_dict = {lot: idx for idx, lot in enumerate(lots)}
# print(lot_index_dict)


[{'LOT': 'A5280140', 'LOT_TYPE': 'Bull'}, {'LOT': 'A5289230', 'LOT_TYPE': 'Cross Qual'}, {'LOT': 'A5280130', 'LOT_TYPE': 'Lead Lot'}, {'LOT': 'A5289210', 'LOT_TYPE': 'Scout 1'}, {'LOT': 'A5289220', 'LOT_TYPE': 'Silent Lot'}, {'LOT': 'A528920A', 'LOT_TYPE': 'SC Scout 2'}, {'LOT': 'A5289200', 'LOT_TYPE': 'SC Scout 1'}, {'LOT': 'A528920B', 'LOT_TYPE': 'EF Scout 1'}, {'LOT': 'A528920C', 'LOT_TYPE': 'EF Scout 2'}]


In [6]:
for lot in lots:
    print(f"LOT: {lot['LOT']}, LOT_TYPE: {lot['LOT_TYPE']}")

LOT: A5280140, LOT_TYPE: Bull
LOT: A5289230, LOT_TYPE: Cross Qual
LOT: A5280130, LOT_TYPE: Lead Lot
LOT: A5289210, LOT_TYPE: Scout 1
LOT: A5289220, LOT_TYPE: Silent Lot
LOT: A528920A, LOT_TYPE: SC Scout 2
LOT: A5289200, LOT_TYPE: SC Scout 1
LOT: A528920B, LOT_TYPE: EF Scout 1
LOT: A528920C, LOT_TYPE: EF Scout 2


In [7]:
prod1.load_lot_list(lots)

LOT: A5280140, LOT_TYPE: Bull
Running master query for LOT: 'A5280140'


d:\Python\NPI\PRODUCT.py:102: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


LOT: A5289230, LOT_TYPE: Cross Qual
Running master query for LOT: 'A5289230'


d:\Python\NPI\PRODUCT.py:102: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


LOT: A5280130, LOT_TYPE: Lead Lot
Running master query for LOT: 'A5280130'


...

d:\Python\NPI\PRODUCT.py:102: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


LOT: A528920A, LOT_TYPE: SC Scout 2
Running master query for LOT: 'A528920A'
LOT: A5289200, LOT_TYPE: SC Scout 1
Running master query for LOT: 'A5289200'


d:\Python\NPI\PRODUCT.py:102: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
d:\Python\NPI\PRODUCT.py:102: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


LOT: A528920B, LOT_TYPE: EF Scout 1
Running master query for LOT: 'A528920B'
LOT: A528920C, LOT_TYPE: EF Scout 2
Running master query for LOT: 'A528920C'


d:\Python\NPI\PRODUCT.py:102: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
d:\Python\NPI\PRODUCT.py:102: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [9]:
test = prod1.LotFlows
